# Example for grasping in Sim
* A grasp motion is designed using KOMO, where the 2-waypoint IK problem defines constraints over a sequence of two configurations, one for each waypoint.
* Each waypoint is here quite rigidly constrained by 6D endeff pose. For convenience, both endeff poses have been defined as 'marker' frames in the configuration. This is a common pattern: Define some convenient reference frames in the configuration, then define NLP constraints relative to these.
* The solver returns a sequence of two joint positions.
* Both are added to the BotOp spline pipeline (with hard timings 2 and 3 sec for them). They define a spline transitioning smoothly through both waypoints.
* The gripper is closed. (In simulation, a hard kinematic link is created!)
* After homing, the gripper is opened. (In simulation, the hard kinematic link is broken again; the object falls.)

In [1]:
import robotic as ry
import numpy as np
import time

In [2]:
ry.params_add({'physx/motorKp': 10000., 'physx/motorKd': 1000., 'physx/angularDamping': 10., 'physx/defaultFriction': 100.})
ry.params_print()

-- module.cpp:operator():95(0) python,
physx/motorKp: 10000,
physx/motorKd: 1000,
physx/angularDamping: 10,
physx/defaultFriction: 100


In [3]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))
C.view(False)

0

In [4]:
#   .setShape(ry.ST.sphere, size=[.025]) \
C.addFrame('box') \
    .setPosition([-.25,.1,.675]) \
    .setShape(ry.ST.ssBox, size=[.05,.05,.05,.005]) \
    .setColor([1,.5,0]) \
    .setMass(.1) \
    .setContact(True)
C.view()

37

In [5]:
# WAYPOINT ENGINEERING:
# manually define frames as an endeff waypoints, relative to box:
way0 = C.addFrame('way0', 'box')
way1 = C.addFrame('way1', 'box')

In [6]:
way0.setShape(ry.ST.marker, size=[.1])
way0.setRelativePose('t(0 0 .1) d(90 0 0 1)')

way1.setShape(ry.ST.marker, size=[.1])
way1.setRelativePose('d(90 0 0 1)')

C.view()

37

In [7]:
# define a 2 waypoint problem in KOMO
komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(2., 1, 5., 0)
komo.addControlObjective([], 0, 1e-0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
komo.addObjective([1.], ry.FS.poseDiff, ['l_gripper', 'way0'], ry.OT.eq, [1e1]);
komo.addObjective([2.], ry.FS.poseDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1]);

In [8]:
ret = ry.NLP_Solver() \
    .setProblem(komo.nlp()) \
    .setOptions( stopTolerance=1e-2, verbose=4 ) \
    .solve()
print(ret)

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):93.1593 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    69.6169  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    47.7566  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    28.1096  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):    11.4588  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    6.02281  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):     4.0943  ACCEPT
--newton-- it:   7  |Delta|:   0.170104  alpha:          1  evals:   8  f(y):    3.57494  ACCEPT
--newton-- it:   8  |Delta|:  0.0579509  alpha:          1  evals:   9  f(y):    3.52862  ACCEPT
--newton-- it:   9  |Delta|:  0.0238862  alpha:          1  evals:  10  f(y

In [9]:
komo.view(False, "waypoints solution")

0

In [10]:
komo.view_close()
path = komo.getPath()

In [11]:
bot = ry.BotOp(C, False)
bot.home(C)

In [12]:
bot.home(C)

In [14]:
bot.gripperMove(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

In [15]:
bot.move(path, [2., 3.])
while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

In [16]:
bot.gripperCloseGrasp(ry._left, 'box')
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

-- kin_physx.cpp:addJoint:299(0) ADDING JOINT l_panda_joint7-box of type rigid with rel [0, 0, 0]


In [17]:
bot.home(C)

In [19]:
bot.gripperMove(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

-- kin_physx.cpp:removeJoint:893(0) REMOVING JOINT 0x3dacab0-0x3daf8d0 of type rigid


In [20]:
del bot

-- bot.cpp:~BotOp:118(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:149(0) shutting down Simulation


In [21]:
del C

## Example for pushing in sim
* A push motion is designed using KOMO, there the 2-waypoint IK problem defines constraints over a sequence of two configurations, one for each waypoint.
* Each waypoint is here quite rigidly constrained by 6D endeff pose. For convenience, both endeff poses have been defined as 'marker' frames in the configuration. This is a common pattern: Define some convenient reference frames in the configuration, then define NLP constraints relative to these.
* The solver returns a sequence of two joint positions.
* Both are added to the BotOp spline pipeline. They define a linear spline interpolation with zero end velocity for both waypoints.

In [22]:
import robotic as ry
import numpy as np
import time

In [23]:
print('ry version', ry.__version__, ry.compiled())
ry.params_add({'physx/motorKp': 10000., 'physx/motorKd': 1000.})
ry.params_print()

ry version 0.1.3 compile time: Jan  4 2024 14:52:25
-- module.cpp:operator():95(0) python,
physx/motorKp: 10000,
physx/motorKd: 1000,
physx/angularDamping: 10,
physx/defaultFriction: 100,
KOMO/verbose: 1,
KOMO/animateOptimization: 0,
KOMO/mimicStable,
KOMO/useFCL,
KOMO/unscaleEqIneqReport!,
KOMO/sampleRate_stable: 0,
opt/verbose: 1,
opt/stopTolerance: 0.01,
opt/stopFTolerance: -1,
opt/stopGTolerance: -1,
opt/stopEvals: 1000,
opt/stopIters: 1000,
opt/stopOuters: 1000,
opt/stopLineSteps: 10,
opt/stopTinySteps: 10,
opt/initStep: 1,
opt/minStep: -1,
opt/maxStep: 0.2,
opt/damping: 1,
opt/stepInc: 1.5,
opt/stepDec: 0.5,
opt/wolfe: 0.01,
opt/boundedNewton,
opt/muInit: 1,
opt/muInc: 5,
opt/muMax: 10000,
opt/muLBInit: 0.1,
opt/muLBDec: 0.2,
opt/maxLambda: -1,
opt/constrainedMethod: ,
seed: 0,
bot/useGripper,
bot/useRobotiq!,
bot/useArm: left,
bot/blockRealRobot!,
botsim/hyperSpeed: 1,
botsim/verbose: 1,
botsim/engine: physx,
physx/verbose: 1,
physx/yGravity!,
physx/softBody!,
physx/multiBody,
p

In [24]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))
C.view(False)

0

In [25]:
C.addFrame('box') \
    .setPosition([-.25,.1,.675]) \
    .setShape(ry.ST.ssBox, size=[.05,.05,.05,.005]) \
    .setColor([1,.5,0]) \
    .setMass(.1) \
    .setContact(True)
C.view()

37

In [26]:
# WAYPOINT ENGINEERING:
# manually define frames as an endeff waypoints, relative to box:
way0 = C.addFrame('way0', 'box')
way1 = C.addFrame('way1', 'box')

In [27]:
way0.setShape(ry.ST.marker, size=[.1])
way0.setRelativePose('t(-.07 0 0) d(90 0 0 1)')

way1.setShape(ry.ST.marker, size=[.1])
way1.setRelativePose('t(.2 0 0) d(90 0 0 1)')

C.view()

37

In [28]:
# define a 2 waypoint problem in KOMO
komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(2., 1, 5., 0)
komo.addControlObjective([], 0, 1e-0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
komo.addObjective([1.], ry.FS.poseDiff, ['l_gripper', 'way0'], ry.OT.eq, [1e1]);
komo.addObjective([2.], ry.FS.poseDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1]);

In [29]:
ret = ry.NLP_Solver() \
    .setProblem(komo.nlp()) \
    .setOptions( stopTolerance=1e-2, verbose=4 ) \
    .solve()
print(ret)

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):100.73 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    75.7925  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    52.4662  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    32.3419  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):    16.2655  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    8.49704  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):    5.68357  ACCEPT
--newton-- it:   7  |Delta|:        0.2  alpha:          1  evals:   8  f(y):    4.63148  ACCEPT
--newton-- it:   8  |Delta|:   0.102645  alpha:          1  evals:   9  f(y):    4.43879  ACCEPT
--newton-- it:   9  |Delta|:  0.0489478  alpha:          1  evals:  10  f(y)

In [30]:
komo.view(False, "waypoints solution")

0

In [31]:
komo.view_close()
path = komo.getPath()

In [32]:
bot = ry.BotOp(C, False)
bot.home(C)

In [33]:
bot.home(C)

In [34]:
bot.gripperClose(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

In [35]:
bot.moveTo(path[0])
bot.moveTo(path[1])
while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

In [37]:
bot.home(C)

In [38]:
bot.gripperMove(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

In [39]:
del bot

-- bot.cpp:~BotOp:118(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:149(0) shutting down Simulation


In [40]:
del C